In [3]:
import pandas as pd
import numpy as np
import PyPDF2
import os
import re
from matplotlib import pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

reader = PyPDF2.PdfReader('./dataset/omfit-2019-2-monthly-servicer-report-september-2023-pdf.pdf')

page4 = reader.pages[3]
page4_text = page4.extract_text()

print(page4_text)

startpoint = page3_text.find('Back-up Servicer Fees') + 21
startpoint

len('Back-up Servicer Fees')

endpoint = page3_text.find(" ", startpoint+1)

float(page3_text[startpoint:endpoint].replace(',','').strip())

len('Interest: \n')

midpoint = page3_text.find('\n', startpoint) + 2
midpoint

endpoint = page3_text.find(" ", midpoint+1)
endpoint

float(page3_text[midpoint:endpoint].replace(',','').strip())

ending_loan_bal = float(page.extract_text()[startpoint:endpoint].replace(',','').strip())

collat_dict = {}
collat_dict['ending_loan_balance'] = [ending_loan_bal]

startpoint = page.extract_text().find('roposed Loans, and Replacement Loans')

midpoint = page.extract_text().find('%', startpoint) + 3

endpoint = page.extract_text().find('%', midpoint+1) - 1

collat_dict['num_loans'] = [float(page.extract_text()[midpoint:endpoint].split()[0].replace(',','').strip())]
collat_dict['wac'] = [float(page.extract_text()[midpoint:endpoint].split()[-1].replace(',','').strip())]
collat_dict['fico'] = [int(page.extract_text()[endpoint+4:endpoint+7])]

startpoint = endpoint + 1

endpoint = page.extract_text().find("Page", startpoint)

collat_dict['wart'] = [float(page.extract_text()[startpoint+7:endpoint-1])]

def get_filename(filename):
    """
    returns name of onemain files
    filename should be a OS dir object"""
    return str(filename)[str(filename).find('\\')+1:str(filename).find(".", 1)].split()[1].replace("'", "")

def create_readers(directory):
    """for the given directory, returns a dict of PyPDF2 readers for each filename in the directory
    dict key is the name of the file
    dict value is the reader
    this function only looks for the naming convention used in  OneMain PDFs"""
    readers = {}
    for filename in os.scandir(directory):
        if filename.is_file():
            name = get_filename(filename)
            readers[name] = PyPDF2.PdfReader(filename.path)
    return readers

directory = './dataset'
collat_dict = {}
collat_dict['ending_loan_balance'] = []
collat_dict['num_loans'] = []
collat_dict['wac'] = []
collat_dict['fico'] = []
collat_dict['wart'] = []
collat_dict['total_prin_coll'] = []
collat_dict['total_int_coll'] = []

readers_dict = create_readers(directory)

dates_pattern = "[0-9]{2}[/][0-9]{2}[/][0-9]{4}"

for key, value in readers_dict.items():
    if 'monthly-servicer-report' in key: #looking at just the trustee reports 
        reader = value
        page2 = reader.pages[1] #second page for loan information
        page2_text = page2.extract_text()
        dates = re.findall(dates_pattern, page2_text)
        payment_date = dates[0]
        end_of_coll_period = dates[1]
        startpoint = page2.extract_text().find('Loan Action Date Loan Principal Balance') + 39
        endpoint = page2.extract_text().find('\n', startpoint) - 1
        ending_loan_bal = float(page2.extract_text()[startpoint:endpoint].replace(',','').strip())            
        collat_dict['ending_loan_balance'].append((payment_date, ending_loan_bal))
        startpoint = page2.extract_text().find('roposed Loans, and Replacement Loans')
        midpoint = page2.extract_text().find('%', startpoint) + 3
        endpoint = page2.extract_text().find('%', midpoint+1) - 1
        collat_dict['num_loans'].append((payment_date, float(page2.extract_text()[midpoint:endpoint].split()[0].replace(',','').strip())))
        collat_dict['wac'].append((payment_date, float(page2.extract_text()[midpoint:endpoint].split()[-1].replace(',','').strip())))
        collat_dict['fico'].append((payment_date, int(page2.extract_text()[endpoint+4:endpoint+7])))
        startpoint = endpoint + 1
        endpoint = page2.extract_text().find("Page", startpoint)
        collat_dict['wart'].append((payment_date, float(page2.extract_text()[startpoint+7:endpoint-1])))
        page3 = reader.pages[2]
        page3_text = page3.extract_text()
        startpoint = page3_text.find('Total Principal Collections') + 28
        midpoint = page3_text.find('\n', startpoint) + 2
        endpoint = page3_text.find(" ", midpoint+1)
        collat_dict['total_prin_coll'].append((payment_date, float(page3_text[midpoint:endpoint].replace(',','').strip())))
        startpoint = page3_text.find('Interest:') + 11
        endpoint = page3_text.find(" ", startpoint+1)
        collat_dict['total_int_coll'] = float(page3_text[startpoint:endpoint].replace(',','').strip())

collat_dict

create_readers('./dataset')


reader = PyPDF2.PdfReader('./dataset/omfit-2019-2-monthly-servicer-report-august-2023-pdf.pdf')
print(filename.path)
page2 = reader.pages[1]
page2_text = page.extract_text()
print(page2_text)
reader = PyPDF2.PdfReader('./dataset/omfit-2019-2-monthly-servicer-report-june-2023-pdf.pdf')
print(filename.path)
page2 = reader.pages[1]
page2_text = page.extract_text()
print(page2_text)

reader = PyPDF2.PdfReader('./dataset/omfit-2019-2-monthly-servicer-report-september-2023-pdf.pdf')
print(filename.path)
page2 = reader.pages[1]
page2_text = page.extract_text()
print(page2_text)

text = "Monthly Servicer Report Collection Period  3010/16/202309/30/2023 Transaction MonthPayment DateEnd DateBeginning Date  4909/01/2023 30/360 Days"

r = "[0-9]{2}[/][0-9]{2}[/][0-9]{4}"
re.findall(r, text)

payment_date = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='G', skiprows=6, nrows=1, header=None, names=['payment_date'])
payment_date = payment_date['payment_date'][0]
payment_date

pool_info_df = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='F', skiprows=55, nrows=5, header=None, names=['end_of_period']).T.reset_index(drop=True)

pool_info_df.columns = ['wac', 'wa_fico', 'wart', 'ending_balance', 'num_loans']
pool_info_df.index = [payment_date]
pool_info_df

prin_coll = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='F', skiprows=77, nrows=1, header=None, names=['total_prin_coll'])
prin_coll.index = [payment_date]
prin_coll

int_coll = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='F', skiprows=82, nrows=1, header=None, names=['total_int_coll'])
int_coll.index = [payment_date]
int_coll

distributions = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='F', skiprows=101, nrows=19, header=None, names=['distributions'])

wf_steps = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='A', skiprows=101, nrows=19, header=None, names=['wf_steps'])

distributions.index = [wf_steps['wf_steps']]

distributions = distributions.T
distributions.index = [payment_date]
distributions

rsv_acct = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='F', skiprows=126, nrows=1, header=None, names=['reserve_acct_bal'])
rsv_acct.index = [payment_date]
rsv_acct

oc = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='F', skiprows=151, nrows=3, header=None, names=['oc'])
oc.index = ['total_oc', 'required_oc', 'in_compliance']
oc = oc.T
oc.index = [payment_date]
oc

events_df = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='A, C:F', skiprows=181, nrows=31, header=None, names=['event', 'initial_pool', 'current_pool', 'limit', 'in_compliance'])
events_df.dropna(axis=0, how='all', inplace=True)
events_df = events_df.reset_index(drop=True)

events_df = events_df[(events_df['event'] != 'AMORTIZATION EVENTS') & (pd.notna(events_df['event']))].reset_index(drop=True)
events_df

In [4]:
directory = "./dataset"
filenames = []
payment_dates = []
for filename in os.scandir(directory):
    if filename.is_file():
        filenames.append(filename)
        payment_date = pd.read_excel(filename, usecols='G', skiprows=6, nrows=1, header=None, names=['payment_date'])
        payment_date = payment_date['payment_date'][0]
        payment_dates.append(payment_date)

In [5]:
files_dates = list(zip(payment_dates, filenames))

In [6]:
files_dates = sorted(files_dates, key = lambda x: x[0])
# files_dates

In [7]:
initial_pool_info_dict = {'bal': {}, 'num_loans': {}, 'wa_fico': {}, 'wac': {}, 'wart': {}}

initial_pool = pd.read_excel(files_dates[0][1], usecols='B:F', skiprows=17, nrows=1, header=None, names=['bal', 'num_loans', 'wa_fico', 'wac', 'wart']).reset_index(drop=True)
initial_pool.index = [files_dates[0][0]]
# initial_pool

In [8]:
initial_bal = float(initial_pool.loc[:, 'bal'])

In [9]:
initial_bal

947374173.76

In [10]:
class_a_coupon = 3.14
class_b_coupon = 3.41
class_c_coupon = 3.66
class_d_coupon = 4.05

In [11]:
bond_info_dict = {}

for i in range(len(files_dates)):
    if files_dates[i][0] <= pd.Timestamp('2021-08-16 00:00:00'):
        row_adjustment = 0
    else:
        row_adjustment = 0
    bonds_info_df = pd.read_excel(files_dates[i][1], usecols='A, B, E, F', skiprows=30+row_adjustment, nrows=4, header=None, names=['bond_name', 'beg_bond_bal', 'end_bond_bal', 'factor'])
    bonds_info_df.dropna(axis=0, how='all', inplace=True)
    bonds_info_df.set_index('bond_name', inplace=True)

    bond_info_dict[files_dates[i][0]] = {(outerKey, innerKey): values for outerKey, innerDict in bonds_info_df.T.to_dict().items() for innerKey, values in innerDict.items()}

In [12]:
bond_df = pd.DataFrame(bond_info_dict)
bond_df = bond_df.T
bond_df.sort_index(inplace=True)

In [13]:
names_list = []
for col in bond_df.columns:
    new_name = f"{col[0]}_{col[1]}"
    names_list.append(new_name)

In [14]:
bond_df.columns = names_list

In [15]:
pool_info_dict = {'beg_bal': {}, 'wac': {}, 'wa_fico': {}, 'wart': {}, 'ending_balance': {}, 'num_loans': {}}

for i in range(len(files_dates)):
    if files_dates[i][0] <= pd.Timestamp('2021-08-16 00:00:00'):
        row_adjustment = 2
    else:
        row_adjustment = 0
    end_period_info_df = pd.read_excel(files_dates[i][1], usecols='F', skiprows=55+row_adjustment, nrows=5, header=None, names=['end_of_period']).T.reset_index(drop=True)
    end_period_info_df.columns = ['wac', 'wa_fico', 'wart', 'ending_balance', 'num_loans']
    end_period_info_df.index = [files_dates[i][0]]
    beg_period_info_df = pd.read_excel(files_dates[i][1], usecols='F', skiprows=39, nrows=1, header=None, names=['beg_bal']).T.reset_index(drop=True)
    beg_period_info_df.columns = ['beg_bal']
    beg_period_info_df.index = [files_dates[i][0]]
    pool_info_df = beg_period_info_df.merge(end_period_info_df, how='outer', left_index=True, right_index=True)
    for key, value in pool_info_df.to_dict().items():
        pool_info_dict[key].update(value)

In [16]:
pool_info_df = pd.DataFrame(pool_info_dict)
pool_info_df.sort_index(inplace=True)

In [17]:
prin_coll_dict = {'total_prin_coll': {}}

for i in range(len(files_dates)):
    if files_dates[i][0] <= pd.Timestamp('2021-08-16 00:00:00'):
        row_adjustment = 2
    else:
        row_adjustment = 0
    prin_coll = pd.read_excel(files_dates[i][1], usecols='F', skiprows=77+row_adjustment, nrows=1, header=None, names=['total_prin_coll'])
    prin_coll.index = [files_dates[i][0]]
    for key, value in prin_coll.to_dict().items():
        prin_coll_dict[key].update(value)

In [18]:
prin_coll = pd.DataFrame(prin_coll_dict)

In [19]:
prin_coll.sort_index(inplace=True)

In [20]:
# prin_coll

In [21]:
int_coll_dict = {'total_int_coll': {}}

for i in range(len(files_dates)):
    if files_dates[i][0] <= pd.Timestamp('2021-08-16 00:00:00'):
        row_adjustment = 2
    else:
        row_adjustment = 0
    int_coll = pd.read_excel(files_dates[i][1], usecols='F', skiprows=82+row_adjustment, nrows=1, header=None, names=['total_int_coll'])
    int_coll.index = [files_dates[i][0]]
    for key, value in int_coll.to_dict().items():
        int_coll_dict[key].update(value)

In [22]:
int_coll = pd.DataFrame(int_coll_dict)

In [23]:
int_coll.sort_index(inplace=True)

In [24]:
# int_coll

In [25]:
wf_dict = {}

counter = 0
for i in range(len(files_dates)):
    if files_dates[i][0] <= pd.Timestamp('2021-08-16 00:00:00'):
        row_adjustment = 2
    else:
        row_adjustment = 0
    distributions = pd.read_excel(files_dates[i][1], usecols='F', skiprows=101+row_adjustment, nrows=19+row_adjustment, header=None, names=['distributions'])
    wf_steps = pd.read_excel(files_dates[i][1], usecols='A', skiprows=101+row_adjustment, nrows=19+row_adjustment, header=None, names=['wf_steps'])
    distributions.index = [wf_steps['wf_steps']]
    distributions = distributions.T
    distributions.index = [payment_dates[i]]
    if counter == 0:
        wf_dict = distributions.to_dict()
    else:
        for key, value in distributions.to_dict().items():
            try:
                wf_dict[key].update(value)
            except KeyError:
                pass
    counter += 1

C:\Users\msimo\AppData\Local\Temp\ipykernel_5480\1050491315.py:17: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  for key, value in distributions.to_dict().items():


In [26]:
wf_df = pd.DataFrame(wf_dict)

In [27]:
wf_df.sort_index(inplace=True)

In [28]:
# wf_df

In [29]:
rsv_acct_dict = {'reserve_acct_bal': {}}

for i in range(len(files_dates)):
    if files_dates[i][0] <= pd.Timestamp('2021-08-16 00:00:00'):
        row_adjustment = 4
    else:
        row_adjustment = 0
    rsv_acct = pd.read_excel(files_dates[i][1], usecols='F', skiprows=126+row_adjustment, nrows=1, header=None, names=['reserve_acct_bal'])
    rsv_acct.index = [files_dates[i][0]]
    for key, value in rsv_acct.to_dict().items():
        rsv_acct_dict[key].update(value)

In [30]:
rsv_df = pd.DataFrame(rsv_acct_dict)
rsv_df.sort_index(inplace=True)

In [31]:
# rsv_df

In [32]:
oc_dict = {'total_oc': {}, 'required_oc': {}, 'in_compliance': {}}

for i in range(len(files_dates)):
    if files_dates[i][0] <= pd.Timestamp('2021-08-16 00:00:00'):
        row_adjustment = 2
        if (files_dates[i][0] == pd.Timestamp('2020-04-14 00:00:00')) or (files_dates[i][0] == pd.Timestamp('2020-05-14 00:00:00')):
            row_adjustment += 2
    else:
        row_adjustment = 0
    oc = pd.read_excel(files_dates[i][1], usecols='F', skiprows=151+row_adjustment, nrows=3, header=None, names=['oc'])
    oc.index = ['total_oc', 'required_oc', 'in_compliance']
    oc = oc.T
    oc.index = [files_dates[i][0]]
    for key, value in oc.to_dict().items():
        oc_dict[key].update(value)

In [33]:
oc_df = pd.DataFrame(oc_dict)
oc_df.sort_index(inplace=True)
# oc_df

In [34]:
events_dict = {}

for i in range(len(files_dates)):
    if files_dates[i][0] <= pd.Timestamp('2021-08-16 00:00:00'):
        row_adjustment = 2
        if (files_dates[i][0] == pd.Timestamp('2020-04-14 00:00:00')) or (files_dates[i][0] == pd.Timestamp('2020-05-14 00:00:00')):
            row_adjustment += 2
    else:
        row_adjustment = 0
    events_df = pd.read_excel(files_dates[i][1], usecols='A, C:F', skiprows=181+row_adjustment, nrows=31, header=None, names=['event', 'initial_pool', 'current_pool', 'limit', 'in_compliance'])
    events_df.dropna(axis=0, how='all', inplace=True)
    events_df = events_df[(events_df['event'] != 'AMORTIZATION EVENTS') & (pd.notna(events_df['event']))].reset_index(drop=True)
    events_df.set_index('event', inplace=True)

    events_dict[files_dates[i][0]] = {(outerKey, innerKey): values for outerKey, innerDict in events_df.T.to_dict().items() for innerKey, values in innerDict.items()}

In [35]:
df = pd.DataFrame(events_dict)
df = df.T
df.sort_index(inplace=True)

In [36]:
# df

In [49]:
combined_df = pool_info_df.merge(prin_coll, how='outer',  left_index=True, right_index=True)

combined_df = combined_df.merge(int_coll, how='outer',  left_index=True, right_index=True)

combined_df = combined_df.merge(rsv_df, how='outer',  left_index=True, right_index=True)

combined_df = combined_df.merge(oc_df, how='outer',  left_index=True, right_index=True)

combined_df = combined_df.merge(bond_df, how='outer',  left_index=True, right_index=True)

In [50]:
combined_df

,beg_bal,wac,wa_fico,wart,ending_balance,num_loans,total_prin_coll,total_int_coll,reserve_acct_bal,total_oc,...,Class A Notes_factor,Class B Notes_beg_bond_bal,Class B Notes_end_bond_bal,Class B Notes_factor,Class C Notes_beg_bond_bal,Class C Notes_end_bond_bal,Class C Notes_factor,Class D Notes_beg_bond_bal,Class D Notes_end_bond_bal,Class D Notes_factor
2019-10-15,9.473742e+08,26.9000,629.00,47.000000,9.474302e+08,126478.0,21238282.59,20422691.08,4500000,47430162.18,...,1,91420000,91420000,1,59210000,59210000,1,98050000,98050000,1
2019-11-14,9.474302e+08,26.8690,628.73,47.143849,9.474250e+08,125787.0,23244260.58,20285276.13,4500000,47424989.11,...,1,91420000,91420000,1,59210000,59210000,1,98050000,98050000,1
2019-12-16,9.474250e+08,26.8148,628.49,47.091538,9.474261e+08,124616.0,21917157.92,20819435.76,4500000,47426092.06,...,1,91420000,91420000,1,59210000,59210000,1,98050000,98050000,1
2020-01-14,9.474261e+08,26.7715,628.29,46.898766,9.474281e+08,124101.0,21558623.72,19454255.41,4500000,47428054.87,...,1,91420000,91420000,1,59210000,59210000,1,98050000,98050000,1
2020-02-14,9.474281e+08,26.7108,628.09,46.575797,9.474294e+08,123814.0,22119150.17,21098722.08,4500000,47429412.48,...,1,91420000,91420000,1,59210000,59210000,1,98050000,98050000,1
2020-03-16,9.474294e+08,26.6160,627.89,46.255265,9.474349e+08,123437.0,23210406.20,20099411.73,4500000,47434890.57,...,1,91420000,91420000,1,59210000,59210000,1,98050000,98050000,1
2020-04-14,9.474349e+08,26.5487,627.77,45.885982,9.474282e+08,123667.0,23284459.01,18897837.31,4500000,47428211.70,...,1,91420000,91420000,1,59210000,59210000,1,98050000,98050000,1
2020-05-14,9.474282e+08,26.1735,627.50,45.929313,9.947993e+08,128450.0,22080028.05,19344946.61,4500000,94799319.07,...,1,91420000,91420000,1,59210000,59210000,1,98050000,98050000,1
2020-06-15,9.947993e+08,25.9223,627.48,45.504034,9.947980e+08,128628.0,25669886.34,20537830.07,4500000,94798041.17,...,1,91420000,91420000,1,59210000,59210000,1,98050000,98050000,1
2020-07-14,9.947980e+08,26.0488,627.68,45.133056,9.947959e+08,128909.0,27359920.41,20736804.90,4500000,94795894.36,...,1,91420000,91420000,1,59210000,59210000,1,98050000,98050000,1


In [51]:
combined_df['rsv_pct_initial'] = combined_df.apply(lambda row: row['reserve_acct_bal']/initial_bal*100, axis=1)

combined_df['rsv_pct_curr'] = combined_df.apply(lambda row: row['reserve_acct_bal']/row['beg_bal']*100, axis=1)

combined_df['total_oc_pct_initial'] = combined_df.apply(lambda row: row['total_oc']/initial_bal*100, axis=1)

combined_df['total_oc_pct_curr'] = combined_df.apply(lambda row: row['total_oc']/row['beg_bal']*100, axis=1)

combined_df['req_oc_initial'] = combined_df.apply(lambda row: row['required_oc']/initial_bal*100, axis=1)

combined_df['req_oc_curr'] = combined_df.apply(lambda row: row['required_oc']/row['beg_bal']*100, axis=1)

combined_df['total_coll'] = combined_df.apply(lambda row: row['total_prin_coll'] + row['total_int_coll'], axis=1)

combined_df['class_a_pct_of_pool'] = combined_df.apply(lambda row: row['Class A Notes_beg_bond_bal'] / row['ending_balance'] * 100, axis=1)

combined_df['class_b_pct_of_pool'] = combined_df.apply(lambda row: row['Class B Notes_beg_bond_bal'] / row['ending_balance'] * 100, axis=1)

combined_df['class_c_pct_of_pool'] = combined_df.apply(lambda row: row['Class C Notes_beg_bond_bal'] / row['ending_balance'] * 100, axis=1)

combined_df['class_d_pct_of_pool'] = combined_df.apply(lambda row: row['Class D Notes_beg_bond_bal'] / row['ending_balance'] * 100, axis=1)

combined_df['bond_wac'] = combined_df.apply(lambda row: ((row['Class A Notes_beg_bond_bal'] * class_a_coupon)+(row['Class B Notes_beg_bond_bal'] * class_b_coupon) + (row['Class C Notes_beg_bond_bal'] * class_c_coupon) + (row['Class D Notes_beg_bond_bal'] * class_d_coupon)) / (row['Class A Notes_beg_bond_bal'] + row['Class B Notes_beg_bond_bal'] + row['Class C Notes_beg_bond_bal'] + row['Class D Notes_beg_bond_bal']), axis=1)

combined_df['excess_spread'] = combined_df.apply(lambda row: row['wac'] - row['bond_wac'], axis = 1)

combined_df['class_a_subordination'] = combined_df.apply(lambda row: row['class_b_pct_of_pool'] + row['class_c_pct_of_pool'] + row['class_d_pct_of_pool'], axis=1)

combined_df['class_b_subordination'] = combined_df.apply(lambda row: row['class_c_pct_of_pool'] + row['class_d_pct_of_pool'], axis=1)

combined_df['class_c_subordination'] = combined_df.apply(lambda row: row['class_d_pct_of_pool'], axis=1)

combined_df['class_a_ce'] = combined_df.apply(lambda row: row['class_a_subordination'] + row['total_oc_pct_curr'] + row['rsv_pct_curr'] + row['excess_spread'], axis=1)

combined_df['class_b_ce'] = combined_df.apply(lambda row: row['class_b_subordination'] + row['total_oc_pct_curr'] + row['rsv_pct_curr'] + row['excess_spread'], axis=1)

combined_df['class_c_ce'] = combined_df.apply(lambda row: row['class_c_subordination'] + row['total_oc_pct_curr'] + row['rsv_pct_curr'] + row['excess_spread'], axis=1)

combined_df['class_d_ce'] = combined_df.apply(lambda row: row['total_oc_pct_curr'] + row['rsv_pct_curr'] + row['excess_spread'], axis=1)

In [52]:
combined_df.reset_index(inplace=True)

In [53]:
combined_df.rename(columns={'index': 'payment_date'}, inplace=True)

In [54]:
combined_df

,payment_date,beg_bal,wac,wa_fico,wart,ending_balance,num_loans,total_prin_coll,total_int_coll,reserve_acct_bal,...,class_d_pct_of_pool,bond_wac,excess_spread,class_a_subordination,class_b_subordination,class_c_subordination,class_a_ce,class_b_ce,class_c_ce,class_d_ce
0,2019-10-15,9.473742e+08,26.9000,629.00,47.000000,9.474302e+08,126478.0,21238282.59,20422691.08,4500000,...,10.349048,3.300776,23.599224,26.247845,16.598585,10.349048,55.328553,45.679293,39.429756,29.080708
1,2019-11-14,9.474302e+08,26.8690,628.73,47.143849,9.474250e+08,125787.0,23244260.58,20285276.13,4500000,...,10.349104,3.300776,23.568224,26.247988,16.598676,10.349104,55.296826,45.647514,39.397942,29.048838
2,2019-12-16,9.474250e+08,26.8148,628.49,47.091538,9.474261e+08,124616.0,21917157.92,20819435.76,4500000,...,10.349092,3.300776,23.514024,26.247958,16.598656,10.349092,55.242742,45.593441,39.343877,28.994785
3,2020-01-14,9.474261e+08,26.7715,628.29,46.898766,9.474281e+08,124101.0,21558623.72,19454255.41,4500000,...,10.349071,3.300776,23.470724,26.247903,16.598622,10.349071,55.199589,45.550307,39.300756,28.951685
4,2020-02-14,9.474281e+08,26.7108,628.09,46.575797,9.474294e+08,123814.0,22119150.17,21098722.08,4500000,...,10.349056,3.300776,23.410024,26.247866,16.598598,10.349056,55.138983,45.489715,39.240173,28.891117
5,2020-03-16,9.474294e+08,26.6160,627.89,46.255265,9.474349e+08,123437.0,23210406.20,20099411.73,4500000,...,10.348996,3.300776,23.315224,26.247714,16.598502,10.348996,55.044602,45.395390,39.145884,28.796888
6,2020-04-14,9.474349e+08,26.5487,627.77,45.885982,9.474282e+08,123667.0,23284459.01,18897837.31,4500000,...,10.349069,3.300776,23.247924,26.247899,16.598619,10.349069,54.976750,45.327470,39.077920,28.728851
7,2020-05-14,9.474282e+08,26.1735,627.50,45.929313,9.947993e+08,128450.0,22080028.05,19344946.61,4500000,...,9.856259,3.300776,22.872724,24.998007,15.808213,9.856259,58.351664,49.161871,43.209917,33.353658
8,2020-06-15,9.947993e+08,25.9223,627.48,45.504034,9.947980e+08,128628.0,25669886.34,20537830.07,4500000,...,9.856272,3.300776,22.621524,24.998039,15.808234,9.856272,57.601279,48.411474,42.459512,32.603240
9,2020-07-14,9.947980e+08,26.0488,627.68,45.133056,9.947959e+08,128909.0,27359920.41,20736804.90,4500000,...,9.856293,3.300776,22.748024,24.998093,15.808268,9.856293,57.727630,48.537805,42.585830,32.729537


In [55]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [63]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=combined_df['payment_date'], y=combined_df['ending_balance'], name="Ending Balance"),
    secondary_y=False,
)
fig.update_traces(marker_color='#ADD8E6')

fig.add_trace(
    go.Scatter(x=combined_df['payment_date'], y=combined_df['total_int_coll'], name="Interest Collections", line_color='blue'),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=combined_df['payment_date'], y=combined_df['total_prin_coll'], name="Principal Collections", line_color='red'),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=combined_df['payment_date'], y=combined_df['total_coll'], name="Total Collections", line_color='green'),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Balance and Collections"
)

# Set x-axis title
fig.update_xaxes(title_text="Payment Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Ending Balance</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Collections</b>", secondary_y=True)

fig.show()

In [58]:
px.line(data_frame=combined_df, x=combined_df['payment_date'], y=combined_df['num_loans'])

In [ ]:
fig, ax = plt.subplots()

sns.barplot(x=combined_df['payment_date'], y=combined_df['ending_balance'], data = combined_df[['payment_date', 'ending_balance']], ax=ax)

ax2 = ax.twinx()

sns.lineplot(x=combined_df['payment_date'], y = combined_df['total_prin_coll'], data = combined_df[['payment_date', 'total_prin_coll']], ax=ax2)
sns.lineplot(x=combined_df['payment_date'], y = combined_df['total_int_coll'], data = combined_df[['payment_date', 'total_int_coll']],ax=ax2)



In [ ]:
plt.figure(figsize = (12,6))

sns.barplot(x=combined_df['payment_date'], y=combined_df['ending_balance'], data = combined_df[['payment_date', 'ending_balance']], color='blue')

In [59]:
max_bal = combined_df['ending_balance'].max()+1000000
min_bal = combined_df['ending_balance'].min()-10000000
px.bar(data_frame=combined_df, x=combined_df['payment_date'], y=combined_df['ending_balance'], range_y=[min_bal, max_bal])
